In [ ]:
import pandas as pd
# import matplotlib.pyplot as plt
import yaml
import os
import glob

import sys
sys.path.append('../')
from utils import *


In [ ]:
with open("../config.yaml", "r") as yamlfile:
    cfg = yaml.safe_load(yamlfile)

for section in cfg:
    RANDOM_STATE_SEED = cfg["variables"]["RANDOM_STATE_SEED"]
    THRESHOLD = cfg["variables"]["THRESHOLD"]
    THRESHOLD_LOSS = cfg["variables"]["THRESHOLD_LOSS"]

    PROCESSED_DATA_DIR = cfg["data_path"]["PROCESSED_DATA_DIR"]

DATA_DIR = 'C:/Users/Admin/Desktop/DATN/DATA/Unraveled_DATA'

In [ ]:
dataset_paths = [os.path.join(DATA_DIR,  dataset_path) for dataset_path in os.listdir(DATA_DIR)]
all_csv_files = []
for dataset_path in dataset_paths:
    csv_files_directory = glob.glob(dataset_path + '/*.csv')
    all_csv_files.extend(csv_files_directory)

In [ ]:
# all_dataframes = {}

if not os.path.exists(PROCESSED_DATA_DIR):
    os.mkdir(PROCESSED_DATA_DIR)

In [ ]:
# for csv_file in all_csv_files:
#     convert_csv_to_dataframe(csv_file)

In [ ]:
print(f'Number of apt2021 files: {len(all_csv_files)}')

In [ ]:
apt_files = []
def get_apt_files(csv_file, threshold):
    df = pd.read_csv(csv_file)
    if (df['Stage'].value_counts()['Benign'] / len(df)) * 100 < threshold:
        print(f'File: {csv_file}')
        apt_files.append(csv_file)
        print('-------------------------------------------------------------------------')

In [ ]:
for csv_file in all_csv_files:
    get_apt_files(csv_file, threshold=99)

In [ ]:
print(len(apt_files))

In [ ]:
df_temp = [pd.read_csv(file_path) for file_path in all_csv_files]
df = pd.concat(df_temp, ignore_index=True)
print(len(df))

In [ ]:
df = df[df['Stage'] != 'Normal']
print(df['Stage'].value_counts())

In [ ]:
df.value_counts('Stage')

Delete random Benign

In [ ]:
# import numpy as np

# # Get the indices of rows labeled as "Benign"
# benign_indices = df[df['Stage'] == 'Benign'].index

# # Randomly select indices to delete
# indices_to_delete = np.random.choice(benign_indices, 200000, replace=False)

# # Drop rows with selected indices
# df = df.drop(indices_to_delete)

In [ ]:
list_features = [
    'bidirectional_duration_ms', 'src2dst_duration_ms',
    'dst2src_duration_ms', 'src2dst_packets', 'dst2src_packets',
    'src2dst_bytes', 'dst2src_bytes', 'src2dst_max_ps', 'src2dst_min_ps',
    'src2dst_mean_ps', 'src2dst_stddev_ps', 'dst2src_max_ps', 'dst2src_min_ps',
    'dst2src_mean_ps', 'dst2src_stddev_ps', 'bidirectional_mean_piat_ms',
    'bidirectional_stddev_piat_ms', 'bidirectional_max_piat_ms', 
    'bidirectional_min_piat_ms','src2dst_mean_piat_ms', 'src2dst_stddev_piat_ms', 
    'src2dst_max_piat_ms', 'src2dst_min_piat_ms', 'dst2src_mean_piat_ms',
    'dst2src_stddev_piat_ms', 'dst2src_max_piat_ms', 'dst2src_min_piat_ms', 
    'bidirectional_fin_packets', 'src2dst_fin_packets', 'dst2src_fin_packets', 
    'bidirectional_syn_packets', 'src2dst_syn_packets', 'dst2src_syn_packets',
    'bidirectional_rst_packets', 'src2dst_rst_packets', 'dst2src_rst_packets',
    'bidirectional_psh_packets', 'src2dst_psh_packets', 'dst2src_psh_packets', 
    'bidirectional_ack_packets', 'src2dst_ack_packets', 'dst2src_ack_packets', 
    'bidirectional_urg_packets', 'src2dst_urg_packets', 'dst2src_urg_packets',
    'bidirectional_cwr_packets', 'src2dst_cwr_packets', 'dst2src_cwr_packets', 
    'bidirectional_ece_packets', 'src2dst_ece_packets', 'dst2src_ece_packets',
    'DefenderResponse'
]

df = df[list_features]


Remove column unique value < 6

In [ ]:
# df = pd.read_csv('C:/Users/Admin/Desktop/DATN/DATA/Unraveled_DATA2/W5_D6/net3.csv',on_bad_lines='skip')
cols_unique_val = []

# Dropping features contain unique value for every instance
# for i in range (1,3):
for col in df.columns:
    if df[col].nunique() == 1:
        cols_unique_val.append(col)
        
print(f'Features contain unique value for every instance: {cols_unique_val}')
df.drop(cols_unique_val, axis=1, inplace=True)

Mapping label

In [ ]:

stage_mapping = {
    'Benign': 0,        
    'Reconnaissance': 1,     
    'Establish Foothold': 2,
    'Lateral Movement': 2,
    'Data Exfiltration': 3,
    'Cover up': 3

}

df['Stage'] = df['Stage'].map(stage_mapping)

In [ ]:

stage_mapping = {
    'Benign': 0,        
    'Detected': 1,     
}

df['DefenderResponse'] = df['DefenderResponse'].map(stage_mapping)

In [ ]:
df.value_counts('DefenderResponse')

DATA cleaning

In [ ]:
import numpy as np

# df.value_counts('Stage')
df.replace([-np.inf, np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

# Dropping duplicate samples
df.drop_duplicates(inplace=True)
df.value_counts('DefenderResponse')

In [ ]:
df.to_csv('C:/Users/Admin/Desktop/DATN/DATA/unraveled_defender_response.csv')

-----------------------------------------------
-----------------------------------------------
-----------------------------------------------

In [ ]:
df = pd.read_csv('C:/Users/Admin/Desktop/DATN/DATA/unraveled_defender_response.csv')

In [ ]:
df.columns

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.value_counts('DefenderResponse')

<h2> Training ATTACK</h2>



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from imblearn.over_sampling import SMOTE

In [ ]:

X_train = df.drop('Stage', axis=1)
Y_train = df['Stage']

# x_train = df.iloc[:, :-1].values
# y_train = df.iloc[:, -1].values


x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, stratify=Y_train, test_size=0.1, random_state=42)

smote = SMOTE(random_state=42)

x_train, y_train = smote.fit_resample(x_train, y_train)
x_test, y_test = smote.fit_resample(x_test, y_test)

Random forest

In [ ]:
random_forest_apt2021 = RandomForestClassifier(
    n_estimators=100,
    n_jobs=12,
)

In [ ]:
random_forest_apt2021.fit(x_train, y_train)

In [ ]:
list_ft = ['bidirectional_duration_ms', 'src2dst_duration_ms',
       'dst2src_duration_ms', 'src2dst_packets', 'dst2src_packets',
       'src2dst_bytes', 'dst2src_bytes', 'src2dst_max_ps', 'src2dst_min_ps',
       'src2dst_mean_ps', 'src2dst_stddev_ps', 'dst2src_max_ps',
       'dst2src_min_ps', 'dst2src_mean_ps', 'dst2src_stddev_ps',
       'bidirectional_mean_piat_ms', 'bidirectional_stddev_piat_ms',
       'bidirectional_max_piat_ms', 'bidirectional_min_piat_ms',
       'src2dst_mean_piat_ms', 'src2dst_stddev_piat_ms', 'src2dst_max_piat_ms',
       'src2dst_min_piat_ms', 'dst2src_mean_piat_ms', 'dst2src_stddev_piat_ms',
       'dst2src_max_piat_ms', 'dst2src_min_piat_ms',
       'bidirectional_fin_packets', 'src2dst_fin_packets',
       'dst2src_fin_packets', 'bidirectional_syn_packets',
       'src2dst_syn_packets', 'dst2src_syn_packets',
       'bidirectional_rst_packets', 'src2dst_rst_packets',
       'dst2src_rst_packets', 'bidirectional_psh_packets',
       'src2dst_psh_packets', 'dst2src_psh_packets',
       'bidirectional_ack_packets', 'src2dst_ack_packets',
       'dst2src_ack_packets', 'bidirectional_urg_packets',
       'src2dst_urg_packets']

df_test = pd.read_csv("C:/Users/Admin/Desktop/DATN/DATA/unraveled-test.csv")
df_test = df_test[list_ft]
df_test
df_test = df_test.values
y_test_pred = random_forest_apt2021.predict(df_test)
print(y_test_pred)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = random_forest_apt2021.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)


In [ ]:
df.value_counts('Stage')

In [ ]:
f1 = f1_score(y_test, y_pred, average='weighted')
f2 = f1_score(y_test, y_pred, average='macro')
f3 = f1_score(y_test, y_pred, average='micro')
# accuracy3 = f1_score(y_test, y_pred, average='binary')
pr1 = precision_score(y_test, y_pred, average='weighted')
pr2 = precision_score(y_test, y_pred, average='macro')
pr3 = precision_score(y_test, y_pred, average='micro')

re1 = recall_score(y_test, y_pred, average='weighted')
re2 = recall_score(y_test, y_pred, average='macro')
re3 = recall_score(y_test, y_pred, average='micro')


print(f1)
print(f2)
print(f3)
print("---------------------")
print(pr1)
print(pr2)
print(pr3)
print("---------------------")

print(re1)
print(re2)
print(re3)


# print(accuracy3)

conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

XGBOOST


In [ ]:
import xgboost as xgb
xgb_apt2021 = xgb.XGBClassifier(objective='multi:softmax')

In [ ]:
xgb_apt2021.fit(x_train, y_train)

In [ ]:
y_pred = xgb_apt2021.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

f1 = f1_score(y_test, y_pred, average='weighted')
f2 = f1_score(y_test, y_pred, average='macro')
f3 = f1_score(y_test, y_pred, average='micro')
# accuracy3 = f1_score(y_test, y_pred, average='binary')
pr1 = precision_score(y_test, y_pred, average='weighted')
pr2 = precision_score(y_test, y_pred, average='macro')
pr3 = precision_score(y_test, y_pred, average='micro')

re1 = recall_score(y_test, y_pred, average='weighted')
re2 = recall_score(y_test, y_pred, average='macro')
re3 = recall_score(y_test, y_pred, average='micro')


print("-------------------------------------------------------")
print("f1 : " ,f1)
print("f1 : " ,f2)
print("f1 : " ,f3)
print("-------------------------------------------------------")
print("precision : " , pr1)
print("precision : " , pr2)
print("precision : " , pr3)
print("-------------------------------------------------------")
print("recall : " , re1)
print("recall : " , re2)
print("recall : " , re3)

conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

In [ ]:
from sklearn.model_selection import KFold, cross_val_score


kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
scores = cross_val_score(xgb_scvic, x_train, y_train, cv=kf, scoring='accuracy')

print("Cross-Validation Scores:", scores)
print("Mean Accuracy:", scores.mean())
print("Standard Deviation:", scores.std())

In [ ]:
list_ft = ['bidirectional_duration_ms', 'src2dst_duration_ms',
       'dst2src_duration_ms', 'src2dst_packets', 'dst2src_packets',
       'src2dst_bytes', 'dst2src_bytes', 'src2dst_max_ps', 'src2dst_min_ps',
       'src2dst_mean_ps', 'src2dst_stddev_ps', 'dst2src_max_ps',
       'dst2src_min_ps', 'dst2src_mean_ps', 'dst2src_stddev_ps',
       'bidirectional_mean_piat_ms', 'bidirectional_stddev_piat_ms',
       'bidirectional_max_piat_ms', 'bidirectional_min_piat_ms',
       'src2dst_mean_piat_ms', 'src2dst_stddev_piat_ms', 'src2dst_max_piat_ms',
       'src2dst_min_piat_ms', 'dst2src_mean_piat_ms', 'dst2src_stddev_piat_ms',
       'dst2src_max_piat_ms', 'dst2src_min_piat_ms',
       'bidirectional_fin_packets', 'src2dst_fin_packets',
       'dst2src_fin_packets', 'bidirectional_syn_packets',
       'src2dst_syn_packets', 'dst2src_syn_packets',
       'bidirectional_rst_packets', 'src2dst_rst_packets',
       'dst2src_rst_packets', 'bidirectional_psh_packets',
       'src2dst_psh_packets', 'dst2src_psh_packets',
       'bidirectional_ack_packets', 'src2dst_ack_packets',
       'dst2src_ack_packets', 'bidirectional_urg_packets',
       'src2dst_urg_packets']

df_test = pd.read_csv("C:/Users/Admin/Desktop/DATN/DATA/unraveled-test.csv")
df_test = df_test[list_ft]
df_test
df_test = df_test.values
y_test_pred = xgb_apt2021.predict(df_test)
print(y_test_pred)

In [ ]:
import joblib

# joblib.dump(random_forest_apt2021, 'random_forest_apt2021.pkl')
joblib.dump(xgb_apt2021, 'xgb_apt2021.pkl')

In [ ]:
# # Evaluate the model
# accuracy_before = rf.score(X_test, y_test)
# print(f'Accuracy before feature selection: {accuracy_before:.2f}')

In [ ]:
importances = random_forest_apt2021.feature_importances_
feature_names  = X_train.columns
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

# Rank features by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

In [ ]:
# df = df[df['Stage']=='Data Exfiltration']
# # df = df[df['Stage']=='Establish Foothold']
# # df = df[df['Stage']=='Benign']


# # # Step 3: Plot the data
# plt.figure(figsize=(20, 6))
# plt.scatter(df['bidirectional_min_piat_ms'], df['Stage'], color='skyblue')

# # Add titles and labels
# plt.title('Value by Label')
# plt.xlabel('bidirectional_duration_ms')
# plt.ylabel('Label')

# # Show the plot
# plt.show()

In [ ]:
# print(df['src2dst_bytes'].max())
# print((df['src2dst_bytes'] > 100000).sum())

<h1> TRAINING DEFENDER RESPONSE </h1>

In [ ]:
df = pd.read_csv('C:/Users/Admin/Desktop/DATN/DATA/unraveled_defender_response.csv')


In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline, make_pipeline

In [ ]:
df.columns

In [ ]:
df.value_counts('DefenderResponse')

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.columns

In [ ]:
X_train = df.drop('DefenderResponse', axis=1)
Y_train = df['DefenderResponse']

# x_train = df.iloc[:, :-1].values
# y_train = df.iloc[:, -1].values


x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, stratify=Y_train, test_size=0.1, random_state=42)

smote = SMOTE(random_state=42)

x_train, y_train = smote.fit_resample(x_train, y_train)
x_test, y_test = smote.fit_resample(x_test, y_test)

In [ ]:
models = {
    # 'Logistic_Regression': (
    #     LogisticRegression(),
    #     {
    #         'C': [0.1, 1.0, 10.0, 25.0],
    #         'solver': ['liblinear', 'newton-cg', 'lbfgs', 'sag'],
    #         'max_iter': [100, 200]
    #     }
    # ),
    # 'K Nearest Neighbors': (
    #     KNeighborsClassifier(),
    #     {
    #         'n_neighbors': [5, 25, 50, 100],
    #         'weights': ['uniform', 'distance'],
    #         'p': [1, 2]
    #     }
    # ),
    # 'Support Vector Machine': (
    #     LinearSVC(),
    #     {
    #         'penalty': ['l1', 'l2'],
    #         'loss': ['hinge', 'squared_hinge'],
    #         'C': [0.1, 1.0, 10.0, 25.0],
    #         'max_iter': [1000, 2000]
    #     }
    # ),
    # 'Naive Bayes': (
    #     MultinomialNB(),
    #      {
    #         'alpha': [0.1, 1.0, 10.0],
    #         'fit_prior': [True, False]
    #      }
    # ),

    # 'Decision Tree': (
    #     DecisionTreeClassifier(),
    #     {
    #         'criterion': ['gini', 'entropy', 'log_loss'],
    #         'splitter': ['best', 'random'],
    #         'min_samples_split': [2, 5, 7],
    #         'min_samples_leaf': [1, 3]
    #     }

    # ),
    'Random Forest': (
        RandomForestClassifier(),
        {
            'n_estimators': [50, 100],
            'criterion': ['gini', 'entropy', 'log_loss'],
            'min_samples_split': [2, 5, 7],
            'min_samples_leaf': [1, 3]
        }
    )
}

In [ ]:
def get_best_models(models):

  best_models = {}
  for model_name, model in models.items():
    params = model[1]
    model = model[0]
    try:
      grid_search = GridSearchCV(model, params, cv=5, scoring='accuracy')
      grid_search.fit(x_train, y_train)
      print(model_name)
      print("--------------------------------------------------------------------------")
      print('best score: ', grid_search.best_score_)
      print('best parameters: ', grid_search.best_params_)
      print("--------------------------------------------------------------------------", '\n')
      best_models[model_name] = (grid_search.best_estimator_, grid_search.best_params_)
    except Exception as e:
      print(f'{model_name}: {e}')

  return best_models

In [ ]:
get_best_models(models)

In [ ]:
# import xgboost as xgb
# xgb_def = xgb.XGBClassifier(objective='multi:softmax')
# xgb_def.fit(x_train, y_train)

In [ ]:
random_forest_def = RandomForestClassifier(
    n_estimators=100,
    n_jobs=12,
)

In [ ]:
random_forest_def.fit(x_train, y_train)

In [ ]:
y_pred = random_forest_def.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

f1 = f1_score(y_test, y_pred, average='weighted')
f2 = f1_score(y_test, y_pred, average='macro')
f3 = f1_score(y_test, y_pred, average='micro')
# accuracy3 = f1_score(y_test, y_pred, average='binary')
pr1 = precision_score(y_test, y_pred, average='weighted')
pr2 = precision_score(y_test, y_pred, average='macro')
pr3 = precision_score(y_test, y_pred, average='micro')

re1 = recall_score(y_test, y_pred, average='weighted')
re2 = recall_score(y_test, y_pred, average='macro')
re3 = recall_score(y_test, y_pred, average='micro')


print("-------------------------------------------------------")
print("f1 : " ,f1)
print("f1 : " ,f2)
print("f1 : " ,f3)
print("-------------------------------------------------------")
print("precision : " , pr1)
print("precision : " , pr2)
print("precision : " , pr3)
print("-------------------------------------------------------")
print("recall : " , re1)
print("recall : " , re2)
print("recall : " , re3)

conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

In [ ]:
import joblib

# joblib.dump(random_forest_apt2021, 'random_forest_apt2021.pkl')
joblib.dump(random_forest_def, 'random_forest_unraveled_def.pkl')